In [18]:
!pip install duckdb -q

# Start

In [1]:
%load_ext autoreload
%autoreload 2

# Setup for the Test

In [2]:
from broai.prompt_management.core import Persona, Instructions, Example, Examples
from broai.llm_management.ollama import BedrockOllamaChat
from broai.prompt_management.core import PromptGenerator
from broai.agent_management.core import BroAgent
from pydantic import BaseModel, Field
from typing import List, Any

In [3]:
bedrock_model = BedrockOllamaChat()

In [4]:
class Joke(BaseModel):
    setup:str = Field(description="this is a setup for the joke")
    punchline:str = Field(description="this is a punchline of the joke")

class Jokes(BaseModel):
    jokes:List[Joke]

In [5]:
class InputFormat(BaseModel):
    message:str = Field(description="The user message")

# BroAgent with full Framework: Happy Path

In [6]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Jokes,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request=InputFormat(message="Tell me ten jokes."))
print(response)

jokes=[Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything.'), Joke(setup="Why don't eggs tell jokes?", punchline="They'd crack each other up."), Joke(setup='Why did the tomato turn red?', punchline='Because it saw the salad dressing.'), Joke(setup='What do you call a fake noodle?', punchline='An impasta.'), Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field.'), Joke(setup="Why don't lobsters share?", punchline="Because they're shellfish."), Joke(setup="What do you call a can opener that doesn't work?", punchline="A can't opener."), Joke(setup='I told my wife she was drawing her eyebrows too high.', punchline='She looked surprised.'), Joke(setup="Why don't some couples go to the gym?", punchline="Because some relationships don't work out."), Joke(setup='Why did the bicycle fall over?', punchline='Because it was two-tired.')]
CPU times: user 69 ms, sys: 23.3 ms, total: 92.3 ms
Wall time: 3.04 s


# BroAgent Full Framework: Pydantic Fallback

In [7]:
class Decoy(BaseModel):
    a:str
    b:str
    c:str
    e:int
    f:float

In [8]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

jokes=[Joke(setup='error', punchline='error')]
CPU times: user 65.7 ms, sys: 669 μs, total: 66.3 ms
Wall time: 12.4 s


# BroAgent Full Framework: Default Fallback

In [9]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

unknown error
CPU times: user 62.4 ms, sys: 4.12 ms, total: 66.5 ms
Wall time: 12.5 s


# BroAgent Full Framework: Custom Fallback in string

In [10]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback="This is a custom fallback in string"
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

This is a custom fallback in string
CPU times: user 61.1 ms, sys: 4.21 ms, total: 65.3 ms
Wall time: 12.3 s


# BroAgent Full Framework: Custom Fallback not string

In [11]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions=Instructions(
        instructions=[
            "tell some jokes based on message",
        ],
    ),
    structured_output=Decoy,
    examples=Examples(examples=[
        Example(
            setting="Funny Andy",
            input=InputFormat(message="Gimme three jokes"),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback={"error": "This is another custom fallback"}
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

{'error': 'This is another custom fallback'}
CPU times: user 60.2 ms, sys: 4.99 ms, total: 65.2 ms
Wall time: 12.5 s


# BroAgent with String: Happy Path

In [12]:
%%time
pg = PromptGenerator(
    persona="You are the good bro Andy.",
    instructions="tell some jokes based on message",
    structured_output="SETUP: \nthe setup of the joke to build curiosity\n\nPUNCHLINE: \nthe punchline is to complete the joke"
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request="Tell me ten jokes.")
print(response)

I gotcha! Here are ten jokes for ya:

1. Setup: I told my wife she was drawing her eyebrows too high.
Punchline: She looked surprised.

2. Setup: Why don't scientists trust atoms?
Punchline: Because they make up everything.

3. Setup: Why don't eggs tell jokes?
Punchline: They'd crack each other up.

4. Setup: What do you call a fake noodle?
Punchline: An impasta.

5. Setup: Why did the scarecrow win an award?
Punchline: Because he was outstanding in his field.

6. Setup: What do you call a can opener that doesn't work?
Punchline: A can't opener.

7. Setup: I'm reading a book about anti-gravity.
Punchline: It's impossible to put down.

8. Setup: Why did the bicycle fall over?
Punchline: Because it was two-tired.

9. Setup: What do you call a bear with no socks on?
Punchline: Barefoot.

10. Setup: Why did the banana go to the doctor?
Punchline: Because he wasn't peeling well.
CPU times: user 21.9 ms, sys: 0 ns, total: 21.9 ms
Wall time: 2.33 s


In [13]:
joke_in_string = response

In [14]:
%%time

class InputFormat(BaseModel):
    content:str

pg = PromptGenerator(
    persona="You are a content extractor.",
    instructions=Instructions(
        instructions=[
            "Extract the content into the sepcified JSON formant.",
        ],
    ),
    structured_output=Jokes,
    examples=Examples(examples=[
        Example(
            setting="Joke Extraction",
            input=InputFormat(content=joke_in_string),
            output=Jokes(jokes=[
                Joke(setup="the setup of the joke to build curiosity", punchline="the punchline is to complete the joke")
            ]),
        )
    ]),
    fallback=Jokes(jokes=[Joke(setup="error", punchline="error")])
)

bro = BroAgent(
    prompt_generator=pg,
    model=bedrock_model
)

response = bro.run(request=InputFormat(content=joke_in_string))
print(response)

jokes=[Joke(setup='I told my wife she was drawing her eyebrows too high.', punchline='She looked surprised.'), Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything.'), Joke(setup="Why don't eggs tell jokes?", punchline="They'd crack each other up."), Joke(setup='What do you call a fake noodle?', punchline='An impasta.'), Joke(setup='Why did the scarecrow win an award?', punchline='Because he was outstanding in his field.'), Joke(setup="What do you call a can opener that doesn't work?", punchline="A can't opener."), Joke(setup="I'm reading a book about anti-gravity.", punchline="It's impossible to put down."), Joke(setup='Why did the bicycle fall over?', punchline='Because it was two-tired.'), Joke(setup='What do you call a bear with no socks on?', punchline='Barefoot.'), Joke(setup='Why did the banana go to the doctor?', punchline="Because he wasn't peeling well.")]
CPU times: user 24.1 ms, sys: 139 μs, total: 24.3 ms
Wall time: 2.93 s


# DuckStore

In [15]:
from broai.duckdb_management.utils import get_create_table_query, get_insert_query, get_batch_update_query, get_where_statement
from broai.duckdb_management.interface import DuckStoreInterface

In [16]:
schemas = {
    "doc_id": "VARCHAR",
    "content": "VARCHAR",
    "data": "JSON"
}

sm = DuckStoreInterface(db="./duckmemory.db", table="sessionmemory", schemas=schemas)

In [17]:
sm.create_table()

In [18]:
sm.sql_df(query="SELECT * FROM sessionmemory;")

,doc_id,content,data


In [19]:
# sm.remove_database(confirm="remove database")

In [20]:
# sm.drop_table()

In [21]:
sm.show_schemas()

{'doc_id': 'VARCHAR', 'content': 'VARCHAR', 'data': 'JSON'}

In [22]:
_data = [
    ["0", "a"],
    ["1", "b"]
]
data = ", ".join([f"('{d[0]}', '{d[1]}')" for d in _data])
sm.add(fields=["doc_id", "content"], data=data)

In [23]:
sm.read(fields=["*"])

,doc_id,content,data
0,0,a,None
1,1,b,None


In [24]:
_data = [
    ["0", "aa"],
    ["1", "bb"]
]
data = ", ".join([f"('{d[0]}', '{d[1]}')" for d in _data])
sm.update(schemas={"doc_id": "VARCHAR", "content": "VARCHAR"}, data=data, ref_keys=["doc_id"])

In [25]:
sm.read()

,doc_id,content,data
0,0,aa,None
1,1,bb,None


# Utility

In [26]:
from broai.interface import Context, TaskStatus

In [27]:
Context(context="Test")

Context(id='92ddf3f1-c3d4-47de-8fa9-cd3569eae28f', context='Test', metadata=None, type='document', created_at='2025-04-16 23:52:02.314933')

In [28]:
TaskStatus.NOT_STARTED

<TaskStatus.NOT_STARTED: 'not_started'>

In [29]:
TaskStatus.PENDING

<TaskStatus.PENDING: 'pending'>

In [30]:
TaskStatus.DONE

<TaskStatus.DONE: 'done'>